<a href="https://colab.research.google.com/github/G750cloud/20MA573/blob/master/Final_Project_Value_American_put_option_by_LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Asian Put Option#
import numpy as np
import pandas as pd
import scipy

def L0(x):
    ret = np.exp(-x/2)
    ret[ret == 1] = 0
    return ret

def L1(x):
    ret = np.exp(-x/2) * (1-x)
    ret[ret == 1] = 0
    return ret

def L2(x):
    ret = np.exp(-x/2) * (1 - 2*x + (x**2)/2)
    ret[ret == 1] = 0
    return ret

def linear(x1,x2,x3,a1,a2,a3):
    return x1*a1 + x2*a2 + x3*a3

def fit_reg_ls(x, y):
    l0 = L0(x)
    l1 = L1(x)
    l2 = L2(x)

    x_stack = np.stack((l0, l1, l2), axis=1)

    a1, a2, a3 = np.linalg.lstsq(x_stack, y)[0]

    return linear(l0,l1,l2,a1,a2,a3)

def payoff(S, K):
    payoff = K-S
    payoff[payoff < 0] = 0
    return payoff

def determine_exercise(S, curr_payoff_matrix, delta, r, K):
    padding = curr_payoff_matrix.shape[1]
    discounts = 0.94176

    St = np.array([v if K-v > 0 else 0 for v in S])
    Pt_related = np.asarray([curr_payoff_matrix[i,:] if v > 0 else np.zeros(padding) for i, v in enumerate(St)])
    y = np.sum(Pt_related * discounts, axis=1)

    y_hat = fit_reg_ls(St, y)
    Pt = payoff(S, K)
    P_continuation = y_hat

    payoff_t = np.array([])
    for i in range(len(Pt)):
        if Pt[i] > P_continuation[i]:
            # Immediately exercise
            payoff_t = np.append(payoff_t, Pt[i])

            # Erase previously added payoff
            curr_payoff_matrix[i,:][curr_payoff_matrix[i,:] > 0] = 0

        else:
            # Do not exercise, keep it for future
            payoff_t = np.append(payoff_t, 0)

    curr_payoff_matrix = np.c_[payoff_t.reshape(-1,1), curr_payoff_matrix]

    return curr_payoff_matrix

def backward_computation(S_paths, r, K, T, delta):
    
    # Amount of time intervals
    N = T/delta

    # Backward looping
    for n in np.arange(N,0,-1):
        n = int(n)
        print('..... Currently at timestep {} ......'.format(n))
        S = S_paths[:,n]
        if n == N:
            # At expiration, payoff matrix would be the immediate exercise value at T
            Mpayoff = payoff(S, K).reshape(-1,1)
        else:
            Mpayoff = determine_exercise(S, Mpayoff, delta, r, K)

    return Mpayoff

def valuation(payoff, delta, r):
    
    M, N = payoff.shape
    discounts = np.exp([-(i+1)*r*delta for i in range(N)])

    value = np.sum(payoff * discounts)/M

    return (value)

S_paths = np.array([[1,1.09,1.08,1.34],[1,1.16,1.26,1.54],[1,1.22,1.07,1.03],[1,0.93,0.97,0.92],[1,1.11,1.56,1.52],[1,0.76,0.77,0.9],[1,0.92,0.84,1.01],[1,0.88,1.22,1.34]])
r = 0.06
K = 1.1
T = 3
delta = 1

aa = backward_computation(S_paths, r, K, T, delta)
print(aa)
valuation(aa, 1, 0.06)

..... Currently at timestep 3 ......
..... Currently at timestep 2 ......
..... Currently at timestep 1 ......
[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.07]
 [0.17 0.   0.  ]
 [0.   0.   0.  ]
 [0.34 0.   0.  ]
 [0.18 0.   0.  ]
 [0.22 0.   0.  ]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


0.11443433004505697